# Let's go PRO!

Advanced RAG Techniques!

Let's start by digging into ingest:

1. No LangChain! Just native for maximum flexibility
2. Let's use an LLM to divide up chunks in a sensible way
3. Let's use the best chunk size and encoder from yesterday
4. Let's also have the LLM rewrite chunks in a way that's most useful ("document pre-processing")

In [1]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm
from litellm import completion
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import gradio as gr


load_dotenv(override=True)

MODEL = "gpt-4.1-nano"

DB_NAME = "cihp_preprocessed_db"
collection_name = "docs"
embedding_model = "text-embedding-3-large"
KNOWLEDGE_BASE_PATH = Path("knowledge-base-cihp")
AVERAGE_CHUNK_SIZE = 500

openai = OpenAI()

In [2]:
# Inspired by LangChain's Document - let's have something similar

class Result(BaseModel):
    page_content: str
    metadata: dict

In [3]:
# A class to perfectly represent a chunk

class Chunk(BaseModel):
    headline: str = Field(description="A brief heading for this chunk, typically a few words, that is most likely to be surfaced in a query")
    summary: str = Field(description="A few sentences summarizing the content of this chunk to answer common questions")
    original_text: str = Field(description="The original text of this chunk from the provided document, exactly as is, not changed in any way")

    def as_result(self, document):
        metadata = {"source": document["source"], "type": document["type"]}
        return Result(page_content=self.headline + "\n\n" + self.summary + "\n\n" + self.original_text,metadata=metadata)


class Chunks(BaseModel):
    chunks: list[Chunk]

## Three steps:

1. Fetch documents from the knowledge base, like LangChain did
2. Call an LLM to turn documents into Chunks
3. Store the Chunks in Chroma

That's it!

### Let's start with Step 1

In [4]:
def fetch_documents():
    """A homemade version of the LangChain DirectoryLoader"""

    documents = []

    for folder in KNOWLEDGE_BASE_PATH.iterdir():
        doc_type = folder.name
        for file in folder.rglob("*.md"):
            with open(file, "r", encoding="utf-8") as f:
                documents.append({"type": doc_type, "source": file.as_posix(), "text": f.read()})

    print(f"Loaded {len(documents)} documents")
    return documents

In [5]:
documents = fetch_documents()

Loaded 10 documents


### Donezo! On to Step 2 - make the chunks

In [6]:
def make_prompt(document):
    how_many = (len(document["text"]) // AVERAGE_CHUNK_SIZE) + 1
    return f"""
You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

{document["text"]}

Respond with the chunks.
"""

In [7]:
print(make_prompt(documents[0]))


You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: coolgirls
The document has been retrieved from: knowledge-base-cihp/coolgirls/Cool girls concept note for evaluation .docx.md

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into 20 chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

Out

In [8]:
def make_messages(document):
    return [
        {"role": "user", "content": make_prompt(document)},
    ]

In [9]:
make_messages(documents[0])

[{'role': 'user',
  'content': "\nYou take a document and you split the document into overlapping chunks for a KnowledgeBase.\n\nThe document is from the shared drive of a company called Insurellm.\nThe document is of type: coolgirls\nThe document has been retrieved from: knowledge-base-cihp/coolgirls/Cool girls concept note for evaluation .docx.md\n\nA chatbot will use these chunks to answer questions about the company.\nYou should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.\nThis document should probably be split into 20 chunks, but you can have more or less as appropriate.\nThere should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.\n\nFor each chunk, you should provide a headline, a summary, and the original text of the chunk.\nTogether your chunks should represent the entire docum

In [10]:
def process_document(document):
    messages = make_messages(document)
    response = completion(model=MODEL, messages=messages, response_format=Chunks)
    reply = response.choices[0].message.content
    doc_as_chunks = Chunks.model_validate_json(reply).chunks
    return [chunk.as_result(document) for chunk in doc_as_chunks]

In [11]:
process_document(documents[0])

[Result(page_content='Introduction: HIV/AIDS burden among AGYW in Sub-Saharan Africa\n\nThis section outlines the high HIV/AIDS prevalence among adolescent girls and young women (AGYW) in Sub-Saharan Africa, highlighting contributing factors such as socio-cultural barriers, gender-based violence, and lack of empowerment. It emphasizes the need for targeted interventions.\n\n·\xa0\xa0\xa0\xa0\xa0\xa0\xa0*Include the problem statement –*\xa0*what is the program issue or gap to be addressed by the intervention? Highlight who is affected, where, and why is it important to focus on the identified group or sub-population?*\n\n*Sub-Saharan Africa is grappling with a significant HIV/AIDS burden, particularly among adolescent girls and young women (AGYW). Despite comprising only 10% of the total population, AGYW account for about 25% of all HIV infections. UNAIDS reported that 6,200 young women aged 15-24 are infected with HIV on a weekly basis worldwide, highlighting the urgent need for target

In [12]:
def create_chunks(documents):
    chunks = []
    for doc in tqdm(documents):
        chunks.extend(process_document(doc))
    return chunks

In [13]:
chunks = create_chunks(documents)

100%|██████████| 10/10 [02:47<00:00, 16.70s/it]


In [14]:
print(len(chunks))

74


### Well that was easy! If a bit slow.

In the python module version, I sneakily use the multi-processing Pool to run this in parallel,
but if you get a Rate Limit Error you can turn this off in the code.

### Finally, Step 3 - save the embeddings

In [15]:
def create_embeddings(chunks):
    chroma = PersistentClient(path=DB_NAME)
    if collection_name in [c.name for c in chroma.list_collections()]:
        chroma.delete_collection(collection_name)

    texts = [chunk.page_content for chunk in chunks]
    emb = openai.embeddings.create(model=embedding_model, input=texts).data
    vectors = [e.embedding for e in emb]

    collection = chroma.get_or_create_collection(collection_name)

    ids = [str(i) for i in range(len(chunks))]
    metas = [chunk.metadata for chunk in chunks]

    collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
    print(f"Vectorstore created with {collection.count()} documents")

In [16]:
create_embeddings(chunks)

Vectorstore created with 74 documents


# Nothing more to do here... right?

Wait! Didja think I'd forget??

In [18]:
chroma = PersistentClient(path=DB_NAME)
collection = chroma.get_or_create_collection(collection_name)
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange', 'darkblue'][['coolgirls', 'ndr', 'sop', 'employees', 'coolgirls-radio'].index(t)] for t in doc_types]

In [19]:
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

c:\Users\ukorm\anaconda3\envs\llm-engineer\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [20]:
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

## And now - let's build an Advanced RAG!

We will use these techniques:

1. Reranking - reorder the rank results
2. Query re-writing

In [21]:
class RankOrder(BaseModel):
    order: list[int] = Field(
        description="The order of relevance of chunks, from most relevant to least relevant, by chunk id number"
    )

In [22]:
def rerank(question, chunks):
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance, but you may be able to improve on that.
You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
Reply only with the list of ranked chunk ids, nothing else. Include all the chunk ids you are provided with, reranked.
"""
    user_prompt = f"The user has asked the following question:\n\n{question}\n\nOrder all the chunks of text by relevance to the question, from most relevant to least relevant. Include all the chunk ids you are provided with, reranked.\n\n"
    user_prompt += "Here are the chunks:\n\n"
    for index, chunk in enumerate(chunks):
        user_prompt += f"# CHUNK ID: {index + 1}:\n\n{chunk.page_content}\n\n"
    user_prompt += "Reply only with the list of ranked chunk ids, nothing else."
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    response = completion(model=MODEL, messages=messages, response_format=RankOrder)
    reply = response.choices[0].message.content
    order = RankOrder.model_validate_json(reply).order
    print(order)
    return [chunks[i - 1] for i in order]

In [23]:
RETRIEVAL_K = 10

def fetch_context_unranked(question):
    query = openai.embeddings.create(model=embedding_model, input=[question]).data[0].embedding
    results = collection.query(query_embeddings=[query], n_results=RETRIEVAL_K)
    chunks = []
    for result in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append(Result(page_content=result[0], metadata=result[1]))
    return chunks

In [42]:
question = "What is the SOP on travel advance?"
chunks = fetch_context_unranked(question)

In [43]:
for chunk in chunks:
    print(chunk.page_content[:15]+"...")

Introduction to...
Procurement Req...
Introduction to...
Introduction an...
Introduction an...
Procurement Pro...
Introduction to...
Operational Pro...
Step 5: Conduct...
Step 2: Filling...
Hardware, Softw...
Organization an...
Documentation a...
Procurement of ...
Annual Certific...
Step 1: Develop...
Procurement Mod...
Step 7: Making ...
Step 3: Adverti...
Overview of Sec...


In [29]:
reranked = rerank(question, chunks)

[10, 2, 7, 9, 5, 8, 3, 1, 4, 6]


In [30]:
for chunk in reranked:
    print(chunk.page_content[:15]+"...")

Annual Certific...
Procurement Req...
Operational Pro...
Hardware, Softw...
Procurement Pro...
Introduction to...
Introduction to...
Introduction to...
Introduction an...
Introduction an...


In [ ]:
question = "What are the objectives of the coolgirls program?"
RETRIEVAL_K = 20
chunks = fetch_context_unranked(question)
for index, c in enumerate(chunks):
    if "manchester" in c.page_content.lower():
        print(index)

In [32]:
reranked = rerank(question, chunks)

[1, 3, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [33]:
for index, c in enumerate(reranked):
    if "manchester" in c.page_content.lower():
        print(index)

In [34]:
reranked[0].page_content

'Objectives and Outcomes of the Cool Girls Program\n\nThe program aims to reduce HIV infection rates and improve well-being among AGYW through targeted messaging, mentorship, referrals, and behavioral change strategies, with specific primary and secondary outcomes and indicators.\n\nObjectives and outcomes of the intervention\n\n* *Describe what would be achieved by the intervention, outlining clear and SMART objectives (maybe one or more)*\n\n* Provide targeted messaging on sexual and reproductive health, HIV prevention, treatment adherence, mental well-being, and one-on-one mentoring, along with referrals to services using the cool girls mobile app.\n\n* The Cool Girls Program targets reducing HIV infection rates and preventing transmission among AGYW.\n\n*Specify the****expected****outcomes related to each of the objectives and highlight the primary outcome if more than one outcome*\n\n* *Through bridging knowledge gaps and fostering behavioral change, the program aims to achieve th

In [35]:
def fetch_context(question):
    chunks = fetch_context_unranked(question)
    return rerank(question, chunks)

In [37]:
SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company CIHP.
You are chatting with a user about CIHP SOPs, NDR, Coolgirls Program, Coolgirls Radio, Employees.
Your answer will be evaluated for accuracy, relevance and completeness, so make sure it only answers the question and fully answers it.
If you don't know the answer, say so.
For context, here are specific extracts from the Knowledge Base that might be directly relevant to the user's question:
{context}

With this context, please answer the user's question. Be accurate, relevant and complete.
"""

In [38]:
# In the context, include the source of the chunk

def make_rag_messages(question, history, chunks):
    context = "\n\n".join(f"Extract from {chunk.metadata['source']}:\n{chunk.page_content}" for chunk in chunks)
    system_prompt = SYSTEM_PROMPT.format(context=context)
    return [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]

In [40]:
def rewrite_query(question, history=[]):
    """Rewrite the user's question to be a more specific question that is more likely to surface relevant content in the Knowledge Base."""
    message = f"""
You are in a conversation with a user, answering questions about the company CIHP.
You are about to look up information in a Knowledge Base to answer the user's question.

This is the history of your conversation so far with the user:
{history}

And this is the user's current question:
{question}

Respond only with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content. Focus on the question details.
Don't mention the company name unless it's a general question about the company.
IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
"""
    response = completion(model=MODEL, messages=[{"role": "system", "content": message}])
    return response.choices[0].message.content

In [41]:
rewrite_query("What is the SOP on travel advance?", [])

'Travel advance SOP details'

In [49]:
def answer_question(question: str, history: list[dict] = []) -> tuple[str, list]:
    """
    Answer a question using RAG and return the answer and the retrieved context
    """
    query = rewrite_query(question, history)
    print(query)
    chunks = fetch_context(query)
    messages = make_rag_messages(question, history, chunks)
    response = completion(model=MODEL, messages=messages)
    # return response.choices[0].message.content, chunks

    return response.choices[0].message.content

In [46]:
answer_question("What is the SOP on travel advance?", [])

SOP for travel advance process
[18, 19, 3, 4, 16, 14, 2, 1, 8, 6, 12, 15, 11, 7, 9, 10, 13, 17, 5]


('I do not have information on an SOP specifically for travel advance within the provided CIHP documentation.',
 [Result(page_content="Final Recruitment Steps and Approval\n\nThe HR finalizes the hiring process with references, salary approval, and official employment letters, with the CEO's final approval required.\n\nthe Senior Human Resources Manager also solicits relevant references on the approved candidate prior to hiring decision. The final decision whether to hire a candidate who has been recommended by an interview panel is at the discretion of the Chief Executive Officer.", metadata={'source': 'knowledge-base-cihp/sop/SOP - Recruitment Policy Oct 2012.pdf.md', 'type': 'sop'}),
  Result(page_content='Step 7: Making an Employment Offer and Acceptance\n\nAfter verifying references, the HR manages salary negotiations, prepares the employment offer letter, and confirms acceptance from the candidate.\n\nStep 7: Employment Offer and Acceptance.\n\nOnce at least 2 positive references

In [47]:
answer_question("What are the objectives of the coolgirls program?", [])

Objectives of the Coolgirls Program
[1, 4, 2, 3, 6, 5, 7, 8, 11, 10, 9, 12, 14, 15, 17, 18, 19, 20, 13]


("The objectives of the Cool Girls Program are to reduce HIV infection rates and improve the well-being of adolescent girls and young women (AGYW) through targeted messaging, mentorship, referrals, and behavioral change strategies. Specifically, the program aims to:\n- Provide targeted information on sexual and reproductive health, HIV prevention, treatment adherence, and mental well-being.\n- Offer one-on-one mentoring and referrals to relevant health and social services using the Cool Girls mobile app.\n- Contribute to decreasing HIV infection rates among AGYW.\n- Enhance participants' knowledge and skills related to sexual and reproductive health, gender-based violence, and mental health.\n- Foster behavioral changes that support safer sexual practices, delay in sexual debut, and better prevention of HIV transmission.\n- Promote empowerment and improved well-being among AGYW by creating a safe environment for learning, networking, and personal development.",
 [Result(page_content='O

In [ ]:
gr.ChatInterface(answer_question).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


What is the main focus of CIHP?
[1, 4, 6, 3, 7, 16, 8, 9, 11, 12, 10, 13, 14, 15, 17, 18, 20, 5, 2]
What is the primary mission of CIHP?
[1, 2, 4, 5, 8, 12, 15, 16, 19, 3, 6, 7, 9, 10, 11, 13, 14, 17, 18, 20]
What are the main goals of the CoolGirls program?
[1, 3, 4, 6, 7, 11, 2, 5, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20]
What is the SOP for requesting and processing travel advances?
[14, 3, 17, 2, 7, 6, 1, 4, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19, 20]
What is the company's policy on travel and expenses?
[5, 3, 1, 4, 6, 11, 8, 12, 9, 13, 10, 14, 15, 16, 17, 2, 7, 18, 19, 20]
What is the per diem rate?
[4, 5, 6, 11, 10, 12, 13, 16, 17, 20, 18, 19, 3, 2, 7, 8, 9, 14, 15, 1]
What is CoolGirls Radio?
[1, 2, 4, 3, 5, 6, 8, 7, 9, 10, 11, 13, 15, 17, 14, 16, 18, 19, 20]
Who is Ukor Mvendaga?
[8, 2, 6, 1, 14, 4, 16, 15, 17, 7, 3, 13, 18, 9, 5, 10, 11, 12, 19, 20]
Ukor Mvendaga's position
[18, 19, 20, 14, 16, 9, 4, 3, 1, 2, 5, 6, 7, 8, 10, 11, 12, 13, 15, 17]
Ukor Mvendaga staff ID
[1, 2,